<a href="https://colab.research.google.com/github/Alriyan1/Gen-AI/blob/main/Output_parser_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_google_genai

  Using cached langchain-0.3.4-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os
os.environ["GOOGLE_API_KEY"]="*****************************"

In [4]:
from langchain import PromptTemplate,FewShotPromptTemplate
from langchain_google_genai import GoogleGenerativeAI

In [5]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [6]:
output_parser = CommaSeparatedListOutputParser()

In [7]:
format_instructions = output_parser.get_format_instructions()

In [8]:
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [9]:
prompt = PromptTemplate(
    template="provide 5 examples of {query}.\n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions}
)

In [10]:
llm=GoogleGenerativeAI(temperature=0.9, model="gemini-1.5-flash-002")

In [11]:
prompt=prompt.format(query="Currencies")

In [12]:
print(prompt)

provide 5 examples of Currencies.
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [18]:
output=llm(prompt)
print(output)

US Dollar, Euro, British Pound, Japanese Yen, Canadian Dollar



# json format

In [19]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [20]:
response_schemas =[
    ResponseSchema(name="currency", description="answer to the user's question"),
    ResponseSchema(name="abbrevation", description="Whats the abbrevation of that currency")
]

In [21]:
output_parser=StructuredOutputParser.from_response_schemas(response_schemas)

In [22]:
print(output_parser)

response_schemas=[ResponseSchema(name='currency', description="answer to the user's question", type='string'), ResponseSchema(name='abbrevation', description='Whats the abbrevation of that currency', type='string')]


In [23]:
format_instructions=output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"currency": string  // answer to the user's question
	"abbrevation": string  // Whats the abbrevation of that currency
}
```


In [35]:
prompt=PromptTemplate(
    template="anwer the users question as best as possible.\n{format_instructions}\n{query}",
    input_variables=["query"],
    partial_variables={"format_instructions":format_instructions}
)

In [36]:
print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"currency": string  // answer to the user\'s question\n\t"abbrevation": string  // Whats the abbrevation of that currency\n}\n```'} template='anwer the users question as best as possible.\n{format_instructions}\n{query}'


In [37]:
prompt=prompt.format(query="what is the currency of germany?")

In [38]:
print(prompt)

anwer the users question as best as possible.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"currency": string  // answer to the user's question
	"abbrevation": string  // Whats the abbrevation of that currency
}
```
what is the currency of germany?


In [39]:
output=llm(prompt)
print(output)

```json
{
	"currency": "Euro",
	"abbrevation": "EUR"
}
```

